In [1]:
import tkinter as tk
import random
import numpy as np
import pyaudio
from scipy.fft import fft
from PIL import Image, ImageTk

class FlappyBirdGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Flappy Bird com Frequência Vocal")
        
        self.WIDTH = 800
        self.HEIGHT = 600
        self.BIRD_RADIUS = 10
        self.PIPE_WIDTH = 80
        self.PIPE_GAP = 200
        self.PIPE_SPACING = 300
        
        self.canvas = tk.Canvas(self.root, width=self.WIDTH, height=self.HEIGHT)
        self.canvas.pack()

        # Carregar a imagem de fundo
        self.bg_image = Image.open("background.png")
        self.bg_image = self.bg_image.resize((self.WIDTH, self.HEIGHT), Image.ANTIALIAS)
        self.bg_photo = ImageTk.PhotoImage(self.bg_image)

        # Desenhar a imagem de fundo no canvas
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.bg_photo)

        self.bird = self.canvas.create_oval(50 - self.BIRD_RADIUS, 250 - self.BIRD_RADIUS, 50 + self.BIRD_RADIUS, 250 + self.BIRD_RADIUS, fill="red")

        self.pipes = []

        self.bird_velocity = -1
        self.gravity = 0.7
        self.notabase = 'C4'
        self.freqmin = 261
        
        self.score = 0
        self.score_text = self.canvas.create_text(self.WIDTH // 2, 50, text=f"Pontuação: {self.score}     Nota base: {self.notabase}", font=('Helvetica', 24), fill='white')

        self.init_audio()
        self.start_game()

    def init_audio(self):
        self.CHUNK = 1024
        self.FORMAT = pyaudio.paInt16
        self.CHANNELS = 1
        self.RATE = 44100

        self.p = pyaudio.PyAudio()

        self.stream = self.p.open(format=self.FORMAT,
                                  channels=self.CHANNELS,
                                  rate=self.RATE,
                                  input=True,
                                  frames_per_buffer=self.CHUNK)

    def get_frequency(self, data):
        N = len(data)
        yf = fft(data)
        xf = np.fft.fftfreq(N, 1 / self.RATE)
        idx = np.argmax(np.abs(yf))
        freq = xf[idx]
        return abs(freq)

    def listen(self):
        data = np.frombuffer(self.stream.read(self.CHUNK), dtype=np.int16)
        freq = self.get_frequency(data)
        if freq > self.freqmin:
            self.jump()
        self.root.after(10, self.listen)

    def jump(self):
        self.bird_velocity = -5

    def move_bird(self):
        self.bird_velocity += self.gravity
        self.canvas.move(self.bird, 0, self.bird_velocity)
        self.root.after(30, self.move_bird)

    def create_pipes(self):
        pipe_x = self.WIDTH
        pipe_height = random.randint(100, self.HEIGHT - self.PIPE_GAP - 100)

        pipe_top = self.canvas.create_rectangle(pipe_x, 0, pipe_x + self.PIPE_WIDTH, pipe_height, fill="white")
        pipe_bottom = self.canvas.create_rectangle(pipe_x, pipe_height + self.PIPE_GAP, pipe_x + self.PIPE_WIDTH, self.HEIGHT, fill="white")

        self.pipes.append((pipe_top, pipe_bottom))
        self.root.after(2000, self.create_pipes)

    def move_pipes(self):
        for pipe_top, pipe_bottom in self.pipes:
            self.canvas.move(pipe_top, -10, 0)
            self.canvas.move(pipe_bottom, -10, 0)

            pipe_top_coords = self.canvas.coords(pipe_top)
            if pipe_top_coords[2] < 0:
                self.pipes.remove((pipe_top, pipe_bottom))
                self.canvas.delete(pipe_top)
                self.canvas.delete(pipe_bottom)
                self.score += 1
                self.canvas.itemconfig(self.score_text, text=f"Pontuação: {self.score}     Nota base: {self.notabase}")
                if self.PIPE_GAP > 50:
                    self.PIPE_GAP = self.PIPE_GAP - 5
                if self.score < 10:
                    self.freqmin = 261
                    self.notabase = 'C4'
                elif self.score < 15:
                    self.freqmin = 293
                    self.notabase = 'D4'
                elif self.score < 20:
                    self.freqmin = 329
                    self.notabase = 'E4'
                elif self.score < 25:
                    self.freqmin = 349
                    self.notabase = 'F4'
                elif self.score < 30:
                    self.freqmin = 392
                    self.notabase = 'G4'
                elif self.score < 35:
                    self.freqmin = 440
                    self.notabase = 'A4'
                elif self.score < 40:
                    self.freqmin = 493
                    self.notabase = 'B4'
                    
                    
        self.root.after(30, self.move_pipes)

    def detect_collision(self):
        bird_coords = self.canvas.coords(self.bird)
        for pipe_top, pipe_bottom in self.pipes:
            pipe_top_coords = self.canvas.coords(pipe_top)
            pipe_bottom_coords = self.canvas.coords(pipe_bottom)
            if (bird_coords[2] > pipe_top_coords[0] and bird_coords[0] < pipe_top_coords[2] and
                (bird_coords[1] < pipe_top_coords[3] or bird_coords[3] > pipe_bottom_coords[1])):
                print("Colisão detectada!")
                self.reset_game()
        self.root.after(10, self.detect_collision)

    def reset_game(self):
        self.bird_velocity = -1
        self.canvas.coords(self.bird, 50 - self.BIRD_RADIUS, 250 - self.BIRD_RADIUS, 50 + self.BIRD_RADIUS, 250 + self.BIRD_RADIUS)
        for pipe_top, pipe_bottom in self.pipes:
            self.canvas.delete(pipe_top)
            self.canvas.delete(pipe_bottom)
        self.pipes.clear()
        self.score = 0
        self.canvas.itemconfig(self.score_text, text=f"Pontuação: {self.score}")
        self.PIPE_WIDTH = 80
        self.PIPE_GAP = 200
        self.PIPE_SPACING = 300
        self.notabase = 'C4'
        self.freqmin = 261

    def start_game(self):
        self.move_bird()
        self.create_pipes()
        self.move_pipes()
        self.detect_collision()
        self.listen()

if __name__ == "__main__":
    root = tk.Tk()
    game = FlappyBirdGame(root)
    root.mainloop()


C:\Users\mateu\AppData\Local\Temp\ipykernel_7376\4252592594.py:25: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.bg_image = self.bg_image.resize((self.WIDTH, self.HEIGHT), Image.ANTIALIAS)
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\mateu\anaconda3\lib\tkinter\__init__.py", line 1919, in __call__
    if self.subst:
KeyboardInterrupt


Colisão detectada!


In [2]:
import tkinter as tk
from tkinter import filedialog
import pygame

# Inicializar o pygame mixer
pygame.mixer.init()

def selecionar_arquivo():
    # Abrir uma caixa de diálogo para selecionar o arquivo MIDI
    filepath = filedialog.askopenfilename(
        filetypes=[("MIDI files", "*.mid"), ("All files", "*.*")]
    )
    if filepath:
        label_arquivo.config(text=filepath)
        pygame.mixer.music.load(filepath)
        
def reproduzir(filepath):
    pygame.mixer.music.stop()
    pygame.mixer.music.load(filepath)
    pygame.mixer.music.play()

def pausar():
    pygame.mixer.music.pause()

def continuar():
    pygame.mixer.music.unpause()

def parar():
    pygame.mixer.music.stop()

# Criar a janela principal
root = tk.Tk()
root.title("Reprodutor de MIDI")

# Botão para selecionar o arquivo MIDI
btn_selecionar = tk.Button(root, text="Selecionar Arquivo MIDI", command=selecionar_arquivo)
btn_selecionar.pack()

# Label para mostrar o caminho do arquivo selecionado
label_arquivo = tk.Label(root, text="Nenhum arquivo selecionado")
label_arquivo.pack()

# Botões de controle de reprodução
btn_reproduzir = tk.Button(root, text="Reproduzir", command=reproduzir)
btn_reproduzir.pack()

btn_pausar = tk.Button(root, text="Pausar", command=pausar)
btn_pausar.pack()

btn_continuar = tk.Button(root, text="Continuar", command=continuar)
btn_continuar.pack()

btn_parar = tk.Button(root, text="Parar", command=parar)
btn_parar.pack()

# Iniciar o loop principal do tkinter
root.mainloop()


pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Sua lista de acertos no formato [frequencia, tempo]
acertos = [
    [100, 30],
    [200, 50],
    [150, 20],
    [120, 40]
]

# Encontrar a frequência que levou o maior tempo
maior_tempo = max(acertos, key=lambda x: x[1])
# Encontrar a frequência que levou o menor tempo
menor_tempo = min(acertos, key=lambda x: x[1])

print(f"A frequência que levou o maior tempo é: {maior_tempo[0]} com {maior_tempo[1]} segundos")
print(f"A frequência que levou o menor tempo é: {menor_tempo[0]} com {menor_tempo[1]} segundos")


A frequência que levou o maior tempo é: 200 com 50 segundos
A frequência que levou o menor tempo é: 150 com 20 segundos


In [2]:
import math

def calcular_oitava(frequencia):
    # Calcula a distância em semitons de A4
    distancia_semitons = 12 * math.log2(frequencia / 440)
    
    # Arredonda para o semitom mais próximo
    semitons_mais_proximo = round(distancia_semitons)
    
    # Calcula a oitava corrigida
    oitava = 4 + (semitons_mais_proximo // 12)
    
    return oitava

def verificar_oitavas(frequencias):
    oitavas = [calcular_oitava(freq) for freq, _ in frequencias]
    oitava_min = min(oitavas)
    oitava_max = max(oitavas)
    
    return (oitava_max - oitava_min) > 2

# Exemplo de uso
frequencias = [(261.63, 1.0), (523.25, 0.5), (1046.50, 0.25)]  # C4, C5, C6
oitavas_diferentes = verificar_oitavas(frequencias)
print(f"As notas estão em oitavas muito diferentes: {oitavas_diferentes}")


As notas estão em oitavas muito diferentes: False
